<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/train3_svc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:

train3 = pd.read_csv(r'/content/train_data3.csv')
test = pd.read_csv(r'/content/test_data.csv')

In [3]:

train3['Target'] = train3['Target'].replace(['Others'],'Invalid')


In [4]:
train3['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [5]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)

train3_Xtfidf = tfidf_vect.fit_transform(train3['Sentence'])


In [7]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [8]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [9]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [10]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [11]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [12]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [13]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(train3_Xtfidf,train3['Target'],test_size=0.26,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6758849557522124
                  precision    recall  f1-score   support

        Analysis       0.52      0.39      0.44       127
      Conclusion       0.75      0.59      0.66        46
           Facts       0.74      0.90      0.82       487
         Invalid       0.48      0.45      0.46       128
           Issue       0.59      0.55      0.57        47
Rule/Law/Holding       0.62      0.19      0.29        69

        accuracy                           0.68       904
       macro avg       0.62      0.51      0.54       904
    weighted avg       0.66      0.68      0.65       904



In [14]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [15]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 4, 3, 2, 2, 2, 2, 2, 2, 2, 1, 3, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 3]
[0.9914865851274751, 0.9959548498035405, 0.9954989239079707, 0.9999607282305301, 0.9800596679732891, 0.9872185136480733, 0.999085094152083, 0.9870683810325741, 0.9937200716678327, 0.9915493163809641, 0.9987829334095806, 0.9962894850836316, 0.999437321391852, 0.9976402059564361, 0.9805375341697572, 0.9990528822366402, 0.9922842829059114, 0.9995383286173771, 0.9912471089381804, 0.9931168054511942, 0.9959792572210695, 0.9970863172960114, 0.9986101687639166, 0.9995639897908694, 0.9994214668612877, 0.9908039877783578, 0.9972877869561148, 0.9835060694501871, 0.9870316150981963, 0.9958200026676584, 0.9935657516722162, 0.9929283616604461, 0.9991655875196079, 0.9924510296792453, 0.9908039657076543, 0.9999541328456832, 0.9905318900867327, 0.9848565222454846, 0.9994174769550391, 0.999335767664778, 0.9994127380248599, 0.9966470126718868]
[0, 1, 4, 5, 6, 8, 11, 12, 17, 18, 

In [16]:
unlabel_1=unlabel_1.loc[[0, 1, 4, 5, 6, 8, 11, 12, 17, 18, 19, 21, 23, 24, 26, 29, 30, 31, 33, 35, 37, 39, 40, 45, 48, 49, 50, 51, 52, 55, 56, 59, 62, 70, 72, 76, 77, 87, 89, 94, 96, 98],:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
0,2173,darden sr testified âœi saw staggering â prett...,Facts
1,466,additional moral justification may due complai...,Analysis
4,79,eligible leaves renew contacts family,Facts
5,1955,got downstairs ponce went car wife left home,Facts
6,324,defendant argues impossible reconcile plaintif...,Facts


In [17]:
frame_1 = [train3,unlabel_1]
train3_1 = pd.concat(frame_1)
len(train3_1)

3518

In [18]:
x_train_1 = tfidf_vect.transform(train3_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train3_1['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6732954545454546
                  precision    recall  f1-score   support

        Analysis       0.44      0.40      0.42       101
      Conclusion       0.78      0.55      0.65        38
           Facts       0.75      0.90      0.82       358
         Invalid       0.56      0.51      0.54       111
           Issue       0.62      0.50      0.56        40
Rule/Law/Holding       0.61      0.25      0.35        56

        accuracy                           0.67       704
       macro avg       0.63      0.52      0.55       704
    weighted avg       0.66      0.67      0.65       704



In [19]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2]
[0.9874398573876122, 0.9888052037943267, 0.9992920302133917, 0.9838849155955789, 0.9944938909222721, 0.9838768042092217, 0.9982199324158947, 0.9989097062435136, 0.9999548379612248, 0.9962792743587116, 0.9825007394543985, 0.9988765836601743, 0.9895420358619119, 0.9823036192139865, 0.9994852971713448, 0.9999621765985788, 0.9993797494713874, 0.9995837849193412, 0.9991137077062636, 0.9928972611483198, 0.9986314093720623, 0.9974912047516058, 0.9996123011432101, 0.9995449784732808, 0.9998253529368523, 0.9986447451647019, 0.9988544722750364, 0.9935739342638505, 0.9988282120695924, 0.9922415721396338, 0.9916561830033316, 0.9987967120238446, 0.9981485465086073, 0.9989662558486989, 0.9868465017885274]
[0, 1, 2, 3, 4, 6, 7, 8, 12, 20, 21, 29, 31, 34, 36, 37, 39, 42, 49, 51, 54, 55, 59, 63, 64, 69, 75, 77, 83, 85, 86, 88, 90, 91, 95]
35
35
35


In [20]:
unlabel_2 = unlabel_2.loc[[0, 1, 2, 3, 4, 6, 7, 8, 12, 20, 21, 29, 31, 34, 36, 37, 39, 42, 49, 51, 54, 55, 59, 63, 64, 69, 75, 77, 83, 85, 86, 88, 90, 91, 95],:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [21]:
frame_2 = [train3_1,unlabel_2]
train3_2 = pd.concat(frame_2)
len(train3_2)

3553

In [22]:
x_train_2 = tfidf_vect.transform(train3_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train3_2['Target'],test_size=0.27,random_state=2)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6364583333333333
                  precision    recall  f1-score   support

        Analysis       0.41      0.42      0.42       137
      Conclusion       0.70      0.43      0.53        53
           Facts       0.71      0.88      0.79       483
         Invalid       0.55      0.42      0.48       165
           Issue       0.55      0.28      0.37        58
Rule/Law/Holding       0.57      0.27      0.36        64

        accuracy                           0.64       960
       macro avg       0.58      0.45      0.49       960
    weighted avg       0.62      0.64      0.61       960



In [23]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[0.9899272544366897, 0.9969410906590349, 0.9868093654726349, 0.993067808964534, 0.9996558358165094, 0.9995396672563662, 0.9994222584213771, 0.9871659251248833, 0.9997945565082652, 0.9823207766518421, 0.9890050515610689, 0.9995801069218574, 0.998771637670102, 0.9985063951026427, 0.9870203542492073, 0.9986507233125711, 0.999423285757798, 0.9886165922252395, 0.9989450593261916, 0.9984562181205525, 0.9899341502577361, 0.9892847593482255, 0.9984330445060843, 0.9920277076510116, 0.9910991366900652, 0.9816878701530112, 0.9969040392214736, 0.9990391882130674, 0.999491716335198, 0.9953224775018715, 0.9991829619439352, 0.9964363280077352, 0.9861132040299779, 0.9995681579864016, 0.9808876382020062, 0.9978211711508357, 0.9992013227803117, 0.9856676367076479, 0.9978677569362017, 0.9924255930133836]
[0, 2, 3, 4, 7, 8, 12, 14, 20, 22, 26, 27, 29, 30, 34, 35, 36, 40, 43, 46, 47, 51,

In [24]:
unlabel_3=unlabel_3.loc[[0, 2, 3, 4, 7, 8, 12, 14, 20, 22, 26, 27, 29, 30, 34, 35, 36, 40, 43, 46, 47, 51, 52, 53, 54, 56, 57, 63, 65, 67, 68, 70, 71, 72, 73, 75, 79, 81, 89, 96],:]
frame_3 = [train3_2,unlabel_3]
train3_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train3_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train3_3['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6703755215577191
                  precision    recall  f1-score   support

        Analysis       0.49      0.37      0.42       103
      Conclusion       0.73      0.55      0.63        40
           Facts       0.75      0.88      0.81       375
         Invalid       0.51      0.52      0.52       108
           Issue       0.69      0.51      0.59        43
Rule/Law/Holding       0.46      0.26      0.33        50

        accuracy                           0.67       719
       macro avg       0.61      0.52      0.55       719
    weighted avg       0.65      0.67      0.65       719



In [25]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 3, 2, 2, 3, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[0.9933279501462001, 0.9885932312205883, 0.9900781516563416, 0.998272747701945, 0.9849398454443246, 0.9999338718830075, 0.9882821882118124, 0.9962265352779238, 0.9993792046525596, 0.9953793192713185, 0.9987843181362422, 0.9981691707515983, 0.9957202894414634, 0.9997455994478205, 0.9966354788820451, 0.9996547893588468, 0.9997268050295309, 0.9997777924624514, 0.998526679548604, 0.9968910767122375, 0.9961160616041642, 0.9998611984736615, 0.9969396125421782, 0.9865812129372997, 0.9953575867871961, 0.9941177663885316, 0.9975356902062233, 0.9986361866669844, 0.9964696689464013, 0.994306872352818, 0.994850989722226, 0.9865196329771365, 0.9960699252985459, 0.9929384807613325, 0.9956196999207453, 0.9970682967059414, 0.9999617872061142, 0.9831405800875845, 0.9856105981339616, 0.994913698936974, 0.9804531978654097, 0.9857568605816516, 0.9985848254667147, 0.998466

In [26]:
unlabel_4=unlabel_4.loc[[2, 4, 6, 7, 8, 10, 12, 15, 18, 21, 22, 27, 28, 29, 30, 31, 32, 34, 37, 38, 39, 40, 42, 46, 49, 50, 55, 56, 58, 60, 61, 66, 67, 69, 70, 71, 75, 80, 85, 87, 89, 90, 93, 97, 99 ],:]

In [27]:
frame_4 = [train3_3,unlabel_4]
train3_4 = pd.concat(frame_4)
len(train3_4)
x_train_4 = tfidf_vect.transform(train3_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train3_4['Target'],test_size=0.27,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6734486266531028
                  precision    recall  f1-score   support

        Analysis       0.45      0.36      0.40       138
      Conclusion       0.61      0.56      0.58        50
           Facts       0.78      0.87      0.82       537
         Invalid       0.49      0.51      0.50       144
           Issue       0.52      0.41      0.46        54
Rule/Law/Holding       0.57      0.35      0.43        60

        accuracy                           0.67       983
       macro avg       0.57      0.51      0.53       983
    weighted avg       0.66      0.67      0.66       983



In [28]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 5, 3, 2, 2, 2, 2, 2, 2, 1, 1, 3, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0]
[0.9991737947943414, 0.9973710168841194, 0.994593153248446, 0.9972488979799037, 0.9973019960468623, 0.9976973009157586, 0.9988458973328591, 0.9958130133425959, 0.9949975168541598, 0.992839322286169, 0.9998807321280112, 0.9962345810656477, 0.9850643790389341, 0.9821385650840699, 0.9983011412837863, 0.9992479295320559, 0.993085573909882, 0.9979757232396731, 0.9958562815174293, 0.997940804625171, 0.9898269663901454, 0.9904283915493753, 0.983651312264788, 0.9926954377500967, 0.9999001871868537, 0.9995301151580595, 0.9945877640739705, 0.9981901502905203, 0.9980811416208839, 0.9984334073369161, 0.9819069694995717, 0.9931506496877105, 0.9814636713333119, 0.9948228439738381]
[3, 12, 18, 19, 22, 23, 26, 27, 28, 32, 36, 46, 48, 50, 52, 53, 56, 60, 63, 65, 67, 75, 77, 78, 82, 86, 91, 93, 94, 95, 97, 100, 103, 110]
34
34
34


In [29]:
unlabel_5 = unlabel_5.loc[[3, 12, 18, 19, 22, 23, 26, 27, 28, 32, 36, 46, 48, 50, 52, 53, 56, 60, 63, 65, 67, 75, 77, 78, 82, 86, 91, 93, 94, 95, 97, 100, 103, 110],:]

In [30]:
frame_5 = [train3_4,unlabel_5]
train3_5 = pd.concat(frame_5)
len(train3_5)
x_train_5 = tfidf_vect.transform(train3_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train3_5['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.672108843537415
                  precision    recall  f1-score   support

        Analysis       0.59      0.43      0.49       112
      Conclusion       0.66      0.50      0.57        38
           Facts       0.76      0.89      0.82       383
         Invalid       0.46      0.47      0.46       118
           Issue       0.50      0.35      0.41        34
Rule/Law/Holding       0.62      0.36      0.46        50

        accuracy                           0.67       735
       macro avg       0.60      0.50      0.54       735
    weighted avg       0.66      0.67      0.66       735



In [32]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))

Accuracy 0.7005758157389635


In [33]:
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(test['Target'],test_pred,average='macro'))

(0.6739861827171963, 0.6246173491951051, 0.6389871063296327, None)
